In [1]:
from netgen.occ import * 
from ngsolve import *

from xfem import *
from xfem.mlset import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

DrawDC = MakeDiscontinuousDraw(Draw)

importing ngsxfem-2.1.2504


In [2]:
# physical parameters for Biot
mu  = 1e6
lam = 1e6
# alpha = 1
alpha = 1
K = 1e2 # k^-1
nu = 1
# s0 = 10
s0 = 1e-2

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [3]:
# DG space order
order_eta = 2
order_u = 2
order_p = 1
# penalty parameters
beta_eta = 400
beta_u = 300
# ghost penalty parameters
gamma_s = 0.1
gamma_u = 0.1
gamma_p = 0.5
gamma_m = 0
tau_p = 0

In [4]:
eta_x = 0
eta_y = 0
u_x = 0
u_y = IfPos(y-0.5,x*(x-1),-x*(x-1))
exact_eta = CF((eta_x, eta_y))
exact_u = CF((u_x,u_y))

# 向量形式
fe = CF((0, 0))
fm = CF((0, 0))
fp = 0
etaD = exact_eta
uD = exact_u
uD2 = CF((0, 0))

In [5]:
# Construct background mesh
# Geometry and Mesh
h0 = 1/64
quad_mesh = False

outer = MoveTo(0, 0.2).Rectangle(1, 0.6).Face()
# outer.edges.name="outer"
outer.edges.Max(X).name = "r"
outer.edges.Min(X).name = "l"
outer.edges.Min(Y).name = "b"
outer.edges.Max(Y).name = "t"

inner = MoveTo(0, 0.475).Rectangle(0.75, 0.05).Face()
inner.edges.name="GammaNo"
geo = outer - inner

inner.faces.name="inner"
inner.faces.col = (1, 0, 0)
outer.faces.name="outer"
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=h0))
# Draw(mesh)
# print(mesh.GetBoundaries())

In [6]:
well1 = 1 - 2500*(x-0.2)**2 - 625/4*(y-0.5)**2
well2 = 1 - 2500*(x-0.4)**2 - 625/4*(y-0.5)**2
well3 = 1 - 2500*(x-0.6)**2 - 625/4*(y-0.5)**2
level_sets = (well1,well2,well3)
nr_ls = len(level_sets)
lsets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

for i, lset_p1 in enumerate(lsets_p1):
    InterpolateToP1(level_sets[i],lset_p1)
omega = DomainTypeArray((NEG,NEG,NEG), lsets=lsets_p1, persistent_compress=True)
boundary = omega.Boundary()

In [7]:
# line1 = y - 4*x + 0.895
# line2 = y + 4*x - 2.105
# line3 = y + 4*x - 1.895
# line4 = y - 4*x + 1.105
# level_sets = (line1, line2, line3, line4)
# nr_ls = len(level_sets)
# lsets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

# for i, lset_p1 in enumerate(lsets_p1):
#     InterpolateToP1(level_sets[i],lset_p1)
#     # DrawDC(lset_p1,-1,1,mesh)
    

In [8]:
# omega = ~DomainTypeArray((NEG,NEG,POS,POS), lsets=lsets_p1, persistent_compress=True)
# boundary = omega.Boundary()



# omega = ~DomainTypeArray((NEG,NEG,POS,POS))
# omega.Compress(lsets_p1)
# DrawDC(omega.Indicator(lsets_p1),-1,1,mesh)

# DrawDC(boundary.IndicatorSmoothed(lsets_p1, 0.01), -3.5, 2.5, mesh)
# omega.as_list
# print(boundary.as_list)


In [9]:
mlci = MultiLevelsetCutInfo(mesh,lsets_p1)
els_hasneg = mlci.GetElementsWithContribution(omega)
els_if = mlci.GetElementsWithContribution(boundary)
facets_gp = GetFacetsWithNeighborTypes(mesh,a=els_hasneg,b=els_if,use_and=True)
interior_facets = GetFacetsWithNeighborTypes(mesh,a=els_hasneg,b=els_hasneg,bnd_val_a=False,bnd_val_b=False,use_and=True)

# Draw(BitArrayCF(els_hasneg),mesh)
# Draw(BitArrayCF(els_if),mesh)

els_if_single = {}
for i,dtt in enumerate(boundary):
    els_if_single[dtt] = mlci.GetElementsWithContribution(dtt)
    # Draw(BitArrayCF(els_if_single[dtt]),mesh)

In [10]:
#Unfitted DG spaces
Ehbase = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
# Phbase = L2(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
Phbase = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
E = Compress(Ehbase, GetDofsOfElements(Ehbase, els_hasneg))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, els_hasneg))
P = Compress(Phbase, GetDofsOfElements(Phbase, els_hasneg))
fes = E*U*P
(eta,u,p), (kxi,v,q) = fes.TnT()

In [11]:
h = specialcf.mesh_size
ne = specialcf.normal(mesh.dim) # normal vectors on faces
normals = omega.GetOuterNormals(lsets_p1) # levelsets的外法向量

domega = dCut(lsets_p1,omega,definedonelements=els_hasneg)
# dk = dCut(lsets_p1, omega, skeleton=True, definedonelements=interior_facets)
dk = dx(skeleton=True,definedonelements=interior_facets)
dw = dFacetPatch(definedonelements=facets_gp) # ghost penalty

# Gamma
dsc = {dtt:dCut(lsets_p1,dtt,definedonelements=els_if_single[dtt]) for dtt in boundary}

# Gamma_no
dsn = ds(skeleton=True,definedon=mesh.Boundaries('GammaNo|l|r'))

# Gamma_p outer boundaries
dseta = ds(skeleton=True,definedon=mesh.Boundaries('l|r|t|b'))
dsu = ds(skeleton=True,definedon=mesh.Boundaries('t|b'))

In [12]:
strain_eta = Sym(Grad(eta))
strain_kxi = Sym(Grad(kxi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*ne
mean_stress_kxi = 0.5*(Stress(Sym(Grad(kxi)))+Stress(Sym(Grad(kxi.Other()))))*ne
jump_eta = eta - eta.Other()
jump_kxi = kxi - kxi.Other()


jump_p = p - p.Other()
jump_q = q - q.Other()
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())


mean_dudn = 0.5*(Grad(u)+Grad(u.Other()))*ne
mean_dvdn = 0.5*(Grad(v)+Grad(v.Other()))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [13]:
Ah = BilinearForm(fes)
############## Eq1 ##############
# Ae
Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
        - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta))*dk \
        - (InnerProduct(Stress(Sym(Grad(eta)))*ne,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*ne,eta))*dseta

Ah +=  beta_eta/h*(2*mu*InnerProduct(jump_eta,jump_kxi)+lam*InnerProduct(jump_eta,ne)*InnerProduct(jump_kxi,ne)) * dk
Ah += beta_eta/h*(2*mu*InnerProduct(eta,kxi) + lam*InnerProduct(eta,ne)*InnerProduct(kxi,ne))*dseta

# Be
Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dk - p*kxi*ne*dseta)

# ghost penalty for eta
Ah += gamma_s / (h**2) * (eta - eta.Other()) * (kxi - kxi.Other()) * dw

############## Eq2 ############## 
# Am
Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
        - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
        + K*InnerProduct(u,v)*domega
# Ah += - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*InnerProduct(u,v))*dsn # Gamma_no
Ah += - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*InnerProduct(u,v))*dsu # Gamma_p

un = InnerProduct(u, ne)
vn = InnerProduct(v, ne)
Ah += - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*un*vn)*dsn # Gamma_no
# Ah += - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*un*vn)*dsu # Gamma_p


 # Bm 
Ah += -div(v)*p*domega + mean_p*jump_v*ne*dk + p*v*ne*dsn + p*v*ne*dsu
# Ah += -div(v)*p*domega + mean_p*jump_v*ne*dk + p*v*ne*dsu

# ghost penalty for velocity
Ah += gamma_u / (h**2) * (u - u.Other()) * (v - v.Other()) * dw

############## Eq3 ##############

# M
Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# -Be
Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dk - q*eta*ne*dseta)

# -Bm
Ah += div(u)*q*domega - mean_q*jump_u*ne*dk - q*u*ne*dsn - q*u*ne*dsu
# Ah += div(u)*q*domega - mean_q*jump_u*ne*dk - q*u*ne*dsu


# ghost penalty for pressure 
Ah += gamma_p * (h**3) * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dk

# stabilization
Ah += tau_p*h*jump_p*jump_q*dk
Ah.Assemble()

In [14]:
# r.h.s
lh = LinearForm(fes) 
############## Eq1 ##############
lh += fe*kxi*domega - InnerProduct(etaD,Stress(Sym(Grad(kxi)))*ne)*dseta 
lh += beta_eta/h*(2*mu*InnerProduct(etaD,kxi) + lam*InnerProduct(etaD,ne)*InnerProduct(kxi,ne))*dseta

############## Eq2 ##############
lh += fm*v*domega 
# lh += - (nu*InnerProduct(uD2,Grad(v)*ne) - nu*beta_u/h*uD2*v)*dsn
lh += - (nu*InnerProduct(uD,Grad(v)*ne) - nu*beta_u/h*uD*v)*dsu
lh += - (nu*InnerProduct(uD2,Grad(v)*ne) - nu*beta_u/h*InnerProduct(uD2,ne)*InnerProduct(v,ne))*dsn
# lh += - (nu*InnerProduct(uD,Grad(v)*ne) - nu*beta_u/h*InnerProduct(uD,ne)*InnerProduct(v,ne))*dsu

############## Eq3 ##############
lh += fp*q*domega 
# lh += - alpha*q*etaD*ne*dseta - q*uD2*ne*dsn - q*uD*ne*dsu
lh += - alpha*q*etaD*ne*dseta - q*uD*ne*dsu
lh.Assemble()

In [15]:
gfu = GridFunction(fes)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

In [20]:
mask = omega.Indicator(lsets_p1)
Draw(mask*gfu.components[0],mesh)
# Draw(gfu.components[0],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [17]:
# Draw(mask*gfu.components[1],mesh)
# Draw(gfu.components[1],mesh)

In [21]:
Draw(mask*gfu.components[2],mesh)
# Draw(gfu.components[2],mesh,min=0)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
vtk = VTKOutput(mesh,[gfu.components[0],gfu.components[1],gfu.components[2]],['eta','u','p'],"/mnt/d/ngs_output/Biot_Brinkman/Ex_multistage3",subdivision=0)
vtk.Do() 